<a href="https://colab.research.google.com/github/admenezes/spam-classifier/blob/main/NBspam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program detects if an email is spam (1) or not (0)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import nltk
import sklearn
from nltk.corpus import stopwords
import string


In [ ]:
# Load the data
from google.colab import files
f = files.upload()

Saving enron.csv to enron.csv


In [ ]:
# Read the file
df = pd.read_csv('enron.csv', header = None, delimiter=',')

#Get the column names
df.columns = ['Text','Spam']

# Print first 5 rows
# df[:5,:]
df.head(5)

,Text,Spam
0,Subject: christmas tree farm pictures,0
1,"Subject: vastar resources , inc . gary , produ...",0
2,Subject: calpine daily gas nomination - calpin...,0
3,Subject: re : issue fyi - see note below - alr...,0
4,Subject: meter 7268 nov allocation fyi . - - -...,0


In [ ]:
#Print the shape (Get the number of rows and cols)
df.shape

(4302, 2)

In [ ]:
#Checking for duplicates and removing them
df.drop_duplicates(inplace = True)
#Show the new shape (number of rows & columns)
df.shape

(4059, 2)

In [ ]:
#Show the number of missing (NAN, NaN, na) data for each column
df.isnull().sum()

Text    0
Spam    0
dtype: int64

In [ ]:
#Need to download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Tokenization (a list of tokens), will be used as the analyzer - dividing the entire email into individual words (tokens)
#1.Punctuations are [!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]
#2.Stop words in natural language processing, are useless words (data).
def process_text(text):
    
    #1 Remove Punctuations
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    #2 Remove Stop Words
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    #3 Return a list of clean words
    return clean_words

In [ ]:
#Show the Tokenization (a list of tokens)
df['Text'].head().apply(process_text)

0           [Subject, christmas, tree, farm, pictures]
1    [Subject, vastar, resources, inc, gary, produc...
2    [Subject, calpine, daily, gas, nomination, cal...
3    [Subject, issue, fyi, see, note, already, done...
4    [Subject, meter, 7268, nov, allocation, fyi, f...
Name: Text, dtype: object

In [ ]:
#Convert the text into a matrix of token counts (number of unique words)
from sklearn.feature_extraction.text import CountVectorizer
messages_bow = CountVectorizer(analyzer=process_text).fit_transform(df['Text'])

In [ ]:
#Get the shape of messages_bow
messages_bow.shape

(4059, 40752)

In [ ]:
#Split data into 80% training & 20% testing data sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(messages_bow, df['Spam'], test_size = 0.20, random_state = 0)

In [ ]:
#Testing features and labels
X_test.shape, y_test.shape

((812, 40752), (812,))

In [ ]:
#Training features and labels
X_train.shape, y_train.shape

((3247, 40752), (3247,))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
#Print the first 30 predictions
check1 = classifier.predict(X_train)
print(check1[:30])
#Print the first 30 Actual Label
check2 = y_train.values
print(check2[:30])

[0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0]
[0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0]


In [ ]:
#Evaluate the model on the training data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train ,pred ))
print('Confusion Matrix: \n',confusion_matrix(y_train,pred))
print('\nAccuracy: ', accuracy_score(y_train,pred)) 

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2761
           1       0.99      0.95      0.97       486

    accuracy                           0.99      3247
   macro avg       0.99      0.97      0.98      3247
weighted avg       0.99      0.99      0.99      3247

Confusion Matrix: 
 [[2756    5]
 [  24  462]]

Accuracy:  0.9910686787804127


In [ ]:
#Print the first 30 predictions
check1 = classifier.predict(X_test)
print(check1[:30])
#Print the first 30 Actual Label
check2 = y_test.values
print(check2[:30])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0]


In [ ]:
#Evaluate the model on the test data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_test)
print(classification_report(y_test ,pred ))
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       680
           1       0.98      0.94      0.96       132

    accuracy                           0.99       812
   macro avg       0.98      0.97      0.97       812
weighted avg       0.99      0.99      0.99       812

Confusion Matrix: 
 [[677   3]
 [  8 124]]

Accuracy:  0.9864532019704434


In [ ]:
# CROSS VALIDATION SCORE
from sklearn.model_selection import cross_val_score
scores = cross_val_score(classifier, X_test, y_test, cv=10)

# Print the accuracy of each fold:
print(scores)
print(' ')
# Print the mean accuracy of all 10 folds
print('The average accuracy across all 10 folds is: ',scores.mean())

[0.96341463 0.95121951 0.92592593 0.92592593 0.96296296 0.9382716
 0.97530864 0.96296296 0.98765432 0.96296296]
 
The average accuracy across all 10 folds is:  0.9556609454983439
